## Description：
这个笔记本主要是完成数据集的简单处理与采样工作，全量数据有8个多G的数据，数据量太大，不利于直接使用， 所以想从里面采样出高质量的一些用户数据来做后面模型相关的实验。采样方式如下:
1. 采样前先处理点击日志这个大数据集， 这里的处理主要是删除记录
    1. 先根据时间筛选， 一个12天的数据，这里选择出后7天的拿来用， 具体实现方法： 时间戳转成时间，然后筛选即可
    2. 删除历史记录中的文章不在文章池子里的记录，这些记录即使保留着， 也没有文章画像， 不利于后面的实验， 实现方法， 拼接上文章画像表，然后把文章画像那部分为空记录删除掉，尤其是看不到类别或者发布时间的这种
    3. 删除不合法的点击记录， 即文章的上传时间大于曝光时间的，相当于先曝光，再上传，这种不合法，实现方法：根据时间筛选即可
    4. 删除没有历史点击的用户，即曝光但点击为0的这些用户， 方法: click字段筛选
    5. 剩下的记录， 把观看时间太短的用户删除掉， 只保留3s以上的视频， 小于3s的默认是误点
    6. 简单看下序列长度分布， 序列过长的用户也剔除掉
2. 处理好的日志数据集，从里面采样出2w用户， 由于调试YouTubeDNN， 先快速跑出来， 后面这个可以再加

读入数据的时候， 不能直接读入， 8个多G内存会爆掉，所以需要分块读入，比如一次读10000， 然后这10000条数据，按照上面的四步处理删除，把剩下的记录保存到一个pandas中。<br><br>

下一次运行的时候，就可以直接从处理好的数据里面直接采样即可。

In [1]:
import os
import time
import random
from datetime import datetime

import numpy as np
import pandas as pd

from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

## 导入日志数据

In [2]:
base_path = '全量数据集/off_data'
doc_info_path = os.path.join(base_path, 'doc_info.txt')
train_data_path = os.path.join(base_path, 'train_data.txt')
save_path = 'all_data/off_data.csv'

In [4]:
# 读入doc_info
doc_info = pd.read_csv(doc_info_path, delimiter='\t', names=['article_id', 'title','ctime', 'img_num', 'cat1', 'cat2', 'key_words'])
article_ids = set(doc_info['article_id'])

doc_info['ctime'] = doc_info['ctime'].str.replace('Android', '1625400960000')
doc_info['ctime'].fillna('1625400960000', inplace=True)
doc_info['ctime'] = doc_info['ctime'].apply(lambda x: datetime.fromtimestamp(float(x)/1000) \
                                                        .strftime('%Y-%m-%d %H:%M:%S'))
doc_info['ctime'] = pd.to_datetime(doc_info['ctime'])

## 过滤原始日志数据并写回到文件

In [5]:
def filter_record(chunk, article_ids, doc_info, start_time='2021-06-30 00:00:00'):
    
    # 根据时间筛选， 只用后七天的数据
    chunk['expo_time'] = chunk['expo_time'].apply(lambda x: datetime.fromtimestamp(x/1000) \
                                                        .strftime('%Y-%m-%d %H:%M:%S'))
    chunk['expo_time'] = pd.to_datetime(chunk['expo_time'])
    chunk = chunk[chunk['expo_time'] >= start_time]
    
    # 去掉点击的文章不在总doc里面的记录
    chunk = chunk[chunk['article_id'].isin(article_ids)]
    
    # 拼接上doc的ctime， 然后去掉曝光时间小于上传时间的
    chunk = chunk.merge(doc_info[['article_id', 'ctime']], on='article_id', how='left')
    chunk = chunk[chunk['expo_time'] > chunk['ctime']]
    del chunk['ctime']
    
    # 标签
    chunk = chunk[chunk['click'].isin([0, 1])]
    
    # duration
    chunk = chunk[~((chunk['click']==1) & (chunk['duration']<3))]
    
    return chunk

In [6]:
# 分块读入train_data，并处理每一块
names = ['user_id', 'article_id', 'expo_time', 'net_status', 'flush_nums', 'exop_position', 'click', 'duration']
train_data_reader = pd.read_csv(train_data_path, delimiter='\t', chunksize=100000, iterator=True, names=names)
count = 0
for chunk in tqdm(train_data_reader):
    count += 1
    # print('过滤前形状: ', chunk.shape)
    chunk = filter_record(chunk, article_ids, doc_info)
    # print('过滤后形状: ', chunk.shape)
    if count == 1:
        chunk.to_csv(save_path,index = False)
    else:
        chunk.to_csv(save_path,index = False, mode = 'a',header = False)

1898it [33:07,  1.05s/it]


## 读取新日志数据并采样

In [2]:
new_train_data = pd.read_csv('all_data/off_data.csv')

In [3]:
new_train_data.shape

(63241712, 8)

In [4]:
new_train_data.head()

,user_id,article_id,expo_time,net_status,flush_nums,exop_position,click,duration
0,1000014754,465426190,2021-07-04 15:07:01,5,0,5,0,0
1,1000014754,465815972,2021-07-04 15:07:01,5,0,4,1,285
2,1000014754,465991958,2021-07-04 15:07:01,5,0,0,1,353
3,1000014754,464264603,2021-06-30 08:08:14,5,1,18,0,0
4,1000014754,464140836,2021-06-30 07:35:31,5,0,13,0,0


In [5]:
# 采样之前， 把过短用户的序列删除掉
new_train_data['user_id'].nunique()

1039309

In [6]:
click_df = new_train_data[new_train_data['click']==1]
user_click_count = click_df.groupby('user_id')['click'].apply(lambda x: x.count()).reset_index()

In [7]:
user_click_count.describe()

,user_id,click
count,6.430580e+05,643058.000000
mean,1.937426e+09,13.758264
std,5.209519e+08,21.891624
min,1.734000e+04,1.000000
25%,1.497289e+09,2.000000
50%,2.213135e+09,5.000000
75%,2.402583e+09,16.000000
max,2.447274e+09,1563.000000


In [8]:
user_click_count.head()

,user_id,click
0,17340,46
1,394666,3
2,450280,3
3,456646,1
4,489240,18


In [21]:
# 保留历史行为序列大于等于10的数据  且小于等于100的用户保留下来
stay_users = set(user_click_count[(user_click_count['click'] >= 10) & (user_click_count['click'] <= 100)]['user_id'])

In [24]:
new_train_data = new_train_data[new_train_data['user_id'].isin(stay_users)]

In [25]:
new_train_data.shape

(43928411, 8)

## 数据保存

In [30]:
# 先保存一份处理好的数据, 后面要是再采样， 可以用这份数据采
new_train_data.to_csv('all_data/processed_data.csv')

In [31]:
# 按照用户id采样
user_num = 20000
user_ids = set(new_train_data['user_id'])
sample_users = random.sample(user_ids, user_num)

sample_train_data = new_train_data[new_train_data['user_id'].isin(sample_users)]

In [32]:
sample_train_data.shape

(3939989, 8)

In [34]:
sample_train_data.head()

,user_id,article_id,expo_time,net_status,flush_nums,exop_position,click,duration
1659,1000541010,464467760,2021-06-30 09:57:14,2,0,13,1,28
1660,1000541010,463850913,2021-06-30 09:57:14,2,0,15,0,0
1661,1000541010,464022440,2021-06-30 09:57:14,2,0,17,0,0
1662,1000541010,464586545,2021-06-30 09:58:31,2,1,20,0,0
1663,1000541010,465352885,2021-07-03 18:13:03,5,0,18,0,0


In [35]:
# 保存
sample_train_data.to_csv('all_data/sample_2w_data.csv', index=False)